In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, GlobalMaxPooling2D,Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_datasets = "/kaggle/input/brain-tumor-classification-mri/Training"
validation_datasets = "/kaggle/input/brain-tumor-classification-mri/Testing"

In [3]:
batch_size = 64
image_size = 224
epochs = 3

In [4]:

def prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size):

    train_datasets_generator = ImageDataGenerator(rescale=1./255,
                                                  shear_range = 0.2,
                                                  zoom_range = 0.2,
                                                  horizontal_flip = True,
                                                  fill_mode = "nearest")


    validation_datasets_generator = ImageDataGenerator(rescale=1.0/255)


    train_datasets_generator_data = train_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = train_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )

    validation_datasets_generator_data = validation_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = validation_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )


    return train_datasets_generator_data, validation_datasets_generator_data

In [5]:
train_data , validation_data = prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size)

Found 2870 images belonging to 4 classes.

Found 394 images belonging to 4 classes.


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (image_size, image_size, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation = 'relu'),
    MaxPooling2D(2, 2),

    Flatten(),

    Dense(512, activation = "relu"),
    Dense(4, activation = "softmax")
])

In [7]:
model.compile(optimizer="adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"]
              )

In [8]:
model_checkpoint_filpath = "model_checkpoint.h5"
callbacks_checkpoints = ModelCheckpoint(

    filepath = model_checkpoint_filpath,
    save_weights_only = True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only = True
)

In [9]:
history = model.fit(train_data,
                    steps_per_epoch = len(train_data),
                    epochs = epochs,
                    validation_data = validation_data,
                    validation_steps = len(validation_data),
                    callbacks = [callbacks_checkpoints]
                    )

Epoch 1/3

45/45 [==============================] - 60s 1s/step - loss: 5.9643 - accuracy: 0.4136 - val_loss: 4.9698 - val_accuracy: 0.3655

Epoch 2/3

45/45 [==============================] - 38s 837ms/step - loss: 5.9186 - accuracy: 0.5380 - val_loss: 22.5390 - val_accuracy: 0.3909

Epoch 3/3

45/45 [==============================] - 37s 823ms/step - loss: 16.4964 - accuracy: 0.5829 - val_loss: 22.0311 - val_accuracy: 0.3579


In [10]:
loss, accuracy = model.evaluate(validation_data, batch_size=batch_size)

7/7 [==============================] - 1s 196ms/step - loss: 23.7675 - accuracy: 0.3553


# Apply ResNet101

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense



# Load pre-trained ResNet101 model without the top (fully connected) layers
base_model = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3), pooling=None)

# Freeze the layers of the pre-trained ResNet101 model
for layer in base_model.layers:
    layer.trainable = False

# Build your custom model on top of the pre-trained ResNet101
model = Sequential()
model.add(base_model)
model.add(GlobalMaxPooling2D())  # Use GlobalMaxPooling2D instead of MaxPooling2D
model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Use 'binary' if it is two classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



171446536/171446536 [==============================] - 7s 0us/step

Model: "sequential_1"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  

                                                                 

 global_max_pooling2d (Glob  (None, 2048)              0         

 alMaxPooling2D)                                                 

                                                                 

 dense_2 (Dense)             (None, 512)               1049088   

                                                                 

 dense_3 (Dense)             (None, 4)                 2052      

                                                                 


Total params: 43709316 (166.74 MB)

Trainable params: 1051140 (4.01 MB)

Non-trainable params: 42658176 (162.73 MB)

____________________________________________________

In [12]:
model_checkpoint_filpath3 = "model_checkpoint.h5"
callbacks_checkpoints3 = ModelCheckpoint(

    filepath = model_checkpoint_filpath3,
    save_weights_only = True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only = True
)

In [13]:
history = model.fit(train_data,
                    steps_per_epoch = len(train_data),
                    epochs = epochs,
                    validation_data = validation_data,
                    validation_steps = len(validation_data),
                    callbacks = [callbacks_checkpoints3]
                    )

Epoch 1/3

45/45 [==============================] - 49s 911ms/step - loss: 1.8076 - accuracy: 0.4334 - val_loss: 2.7529 - val_accuracy: 0.3096

Epoch 2/3

45/45 [==============================] - 39s 874ms/step - loss: 1.0980 - accuracy: 0.5328 - val_loss: 2.6002 - val_accuracy: 0.3249

Epoch 3/3

45/45 [==============================] - 38s 851ms/step - loss: 0.9685 - accuracy: 0.5951 - val_loss: 2.6439 - val_accuracy: 0.3604


In [14]:
loss, accuracy = model.evaluate(validation_data, batch_size=batch_size)

7/7 [==============================] - 2s 196ms/step - loss: 2.6439 - accuracy: 0.3604
